<a href="https://colab.research.google.com/github/ktxy2004/PNA_sequence_search/blob/main/Find_match_sequence_in_genome_updated20230515.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Aim to detect sequence mataches to intended PNAs sequence and report whether a translational start site is included. Scan through CDS and exon in a genome. Genome file and corresponding gff file are required.

In [ ]:
##Mount google drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
 !pip install biopython tqdm
from functools import partial
from Bio import SeqIO
from Bio.SeqFeature import FeatureLocation
from Bio.Seq import reverse_complement
from tqdm import tqdm


def find_sequence(genome_file, sequence, gff_file):
    # Load genome sequences
    genome_records = list(SeqIO.parse(genome_file, "fasta"))

    # Create a dictionary of CDS and exon locations keyed by start and end positions
    cds_dict = {}
    with open(gff_file, "r") as f:
        for line in f:
            if not line.startswith("#"):
                cols = line.strip().split("\t")
                feature = cols[2]
                if feature == "CDS" or feature == "exon":
                    start, end = map(int, cols[3:5])
                    strand = cols[6]
                    gene_name = cols[8].split(";")[0].split("=")[1]
                    if feature != "region":
                        cds_dict[(start, end, cols[0])] = (strand, gene_name)

    # Find the locations of the query sequence ## Find exact match
    query_length = len(sequence)
    locations = []
    for genome_record in genome_records:
        genome_seq = genome_record.seq
        chromosome_name = genome_record.id
        for strand, nucleotide_seq in [(+1, sequence), (-1, reverse_complement(sequence))]:
            for i in range(len(genome_seq) - query_length + 1):
                num_matching_bases = sum([genome_seq[i+j].upper() == nucleotide_seq[j].upper() for j in range(query_length)])
                if num_matching_bases == query_length:
                    start_pos = i+1
                    end_pos = i+query_length
                    locations.append((chromosome_name, start_pos, end_pos, strand, nucleotide_seq))

    # Find the CDSs that overlap with the query sequence
    overlapping_cds = []
    for cds_start, cds_end, cds_chromosome in tqdm(cds_dict.keys(), desc="Finding overlapping CDSs"):
        for query_chromosome, query_start, query_end, query_strand, query_seq in locations:
            if cds_start <= query_end and cds_end >= query_start and query_chromosome == cds_chromosome and cds_dict[(cds_start, cds_end, cds_chromosome)][1] != "region":
                if query_strand == 1:
                    if cds_dict[(cds_start, cds_end, cds_chromosome)][0] == "+":
                        strand_comment = "plus plus match"
                    else:
                        strand_comment = "reverse complement match"
                else:
                    if cds_dict[(cds_start, cds_end, cds_chromosome)][0] == "+":
                        strand_comment = "reverse complement match"
                    else:
                        strand_comment = "plus plus match"
                cds_comment = ""
                if query_start <= cds_start <= query_end or query_start <= cds_end <= query_end:
                    cds_comment = "Translational Start Site Included"
                overlapping_cds.append((cds_dict[(cds_start, cds_end, cds_chromosome)][1], cds_start, cds_end, cds_dict[(cds_start, cds_end, cds_chromosome)][0], query_chromosome, query_start, query_end, query_strand, strand_comment, cds_comment, query_seq))



    return overlapping_cds


# Set the filenames and query sequence

genome_file = "/content/drive/MyDrive/Apisum_Buchnera_genome_gff/GCF_000009605.1_ASM960v1_genomic.fna" ##Buchnera strain APS_change the path if necessary
gff_file = "/content/drive/MyDrive/Apisum_Buchnera_genome_gff/GCF_000009605.1_ASM960v1_genomic.gff" ##fasta & gff3 files were downloaded from NCBI Genome and stored in Gdrive
#query_sequence = "GCCATTTGAC" # PNA_GroEL_sequence
#query_sequence = "GCGATTTGTC" # PNA_mm_sequence




def write_results_to_file(results, output_file):
    with open(output_file, "w") as f:
        f.write("Gene Name\tCDS start position\tCDS end position\tStrand\tQuery start position\tQuery end position\tQuery strand\tComment\tExtra Comment\tQuery sequence\n")
        for result in results:
            f.write("\t".join(map(str, result)) + "\n")

# Run the function and write the results to a tab-delimited file
output_file = "output.txt"
results = find_sequence(genome_file, query_sequence, gff_file)
write_results_to_file(results, output_file)



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 6.9 MB/s eta 0:00:00


Finding overlapping CDSs: 100%|██████████| 630/630 [00:00<00:00, 1048992.27it/s]
